# Gradio Chatbot UI for Multi-Agent System
Interact with your AI agents using a conversational interface powered by Gradio and the logic in `mas.py`.

In [1]:
import asyncio
import gradio as gr
from src.mas import Orchestrator

# Initialize the orchestrator
orchestrator = Orchestrator()

# Global chat history
chat_history = []

## Define the Chatbot Logic
This function will be called on each user message. It sends the message to the orchestrator and returns the updated chat history.

In [2]:
def chat_with_agents(user_message, history):
    global chat_history
    if not user_message.strip():
        return history, ""
    # Append user message to history for display
    history = history or []
    history.append(("user", user_message))
    # Run orchestrator and get agent responses
    try:
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        messages = loop.run_until_complete(orchestrator.run(user_message, chat_history))
        # Extract agent responses and update history
        for msg in messages[len(chat_history):]:
            role = getattr(msg, 'role', None) or getattr(msg, 'author_role', None) or 'assistant'
            content = getattr(msg, 'content', str(msg))
            if role == 'user':
                continue
            history.append((role, content))
        chat_history = messages
    except Exception as e:
        history.append(("system", f"Error: {e}"))
    return history, ""

## Build the Gradio UI
This UI includes a chatbot window and a textbox for user input.

In [ ]:
import debugpy

# Allow other clients to attach to debugpy
debugpy.listen(("0.0.0.0", 5678))
print("Waiting for debugger to attach...")

# Pause execution until the debugger is attached
debugpy.wait_for_client()

with gr.Blocks(title="Reasoning over ML - Chatbot") as demo:
    gr.Markdown("""# Multi-Agent Chatbot\nChat with your AI agents below.""")
    chatbot = gr.Chatbot(label="AI Chatbot", height=400)
    textbox = gr.Textbox(placeholder="Type your message and press Enter", show_label=False, lines=1)

    def clear_history():
        global chat_history
        chat_history = []
        return []

    textbox.submit(chat_with_agents, [textbox, chatbot], [chatbot, textbox])
    gr.Button("Clear").click(clear_history, outputs=chatbot)

if __name__ == "__main__":
    demo.launch()

Waiting for debugger to attach...


/tmp/ipykernel_3679391/1310774879.py:12: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="AI Chatbot", height=400)


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


# AuthorRole.USER: 'Help me to interpret the AI model's results'
# AuthorRole.ASSISTANT - AnalystAgent: 'To assist you in interpreting the AI model's results, I will need to retrieve the SHAP values, which are crucial for understanding the contribution of each feature to the model's predictions. Let me fetch the raw SHAP values first.'
# AuthorRole.ASSISTANT - AnalystAgent: 'Based on the retrieved SHAP values, let's break down the analysis into key sections to help you understand the model's predictions:

### Key Influential Features

1. **Sales Rolling Mean (20 days)**
   - **Impact**: This feature has the most significant positive impact on predictions, with SHAP values like `4.216077` and `1.9598305` in some instances. It indicates that recent sales trends over the past 20 days are crucial in driving predictions.
   - **Significance**: A high rolling mean suggests a strong upward trend in sales, which the model interprets as a positive indicator for future sales.

2. **On Promotion*